# Azure OpenAI with LangChain Integration

This notebook demonstrates how to use Azure OpenAI services with LangChain, including authentication using Healthcare Cloud Platform (HCP) tokens.

## Environment Setup

First, load environment variables from a .env file. Refer .env.example


In [ ]:
# Moved functionality to /environment_loading/  See README.md in that folder for details.

from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

## Authentication Functions

Method to fetch token for azure open ai from HCP

In [ ]:
from fx_ai_reusables.authenticators.hcp.interfaces.hcp_authenticator_interface import IHcpAuthenticator
from fx_ai_reusables.authenticators.hcp.concretes.hcp_authenticator import HcpAuthenticator

from fx_ai_reusables.environment_loading.interfaces.azure_llm_config_and_secrets_holder_wrapper_reader_interface import IAzureLlmConfigAndSecretsHolderWrapperReader
from fx_ai_reusables.environment_loading.concretes.azure_llm_config_and_secrets_holder_wrapper_reader import AzureLlmConfigAndSecretsHolderWrapperReader
from fx_ai_reusables.environment_loading.cache_aside_decorators.azure_llm_config_and_secrets_holder_wrapper_cache_aside_decorator import AzureLlmConfigAndSecretsHolderWrapperCacheAsideDecorator
from fx_ai_reusables.configmaps.concretes.env_variable.environment_variables_config_map_retriever import EnvironmentVariablesConfigMapRetriever
from fx_ai_reusables.secrets.concretes.env_variable.environment_variable_secret_retriever import EnvironmentVariableSecretRetriever

config_map_retriever = EnvironmentVariablesConfigMapRetriever()
secrets_retriever = EnvironmentVariableSecretRetriever()

#undecorated is for the raw functionality
undecorated_env_values_reader: IAzureLlmConfigAndSecretsHolderWrapperReader = AzureLlmConfigAndSecretsHolderWrapperReader(
    config_map_retriever, secrets_retriever
)

# now decorate the undecorated reader to provide cache-aside functionality
cache_aside_env_values_reader: IAzureLlmConfigAndSecretsHolderWrapperReader = AzureLlmConfigAndSecretsHolderWrapperCacheAsideDecorator(undecorated_env_values_reader)


#not shown, there is a cache-aside decorator for the HCP authenticator, that looks at the EXPIRES_ON value.
hcp_token_creator:IHcpAuthenticator = HcpAuthenticator(cache_aside_env_values_reader)




## Initialize LLM with Authentication

Create a function to initialize the Azure OpenAI model with HCP token authentication and optional parameters for tools and structured output.

In [ ]:
from fx_ai_reusables.llm.creators.azure_chat_openai_llm_creator import AzureChatOpenAILlmCreator
from fx_ai_reusables.llm.creators.interfaces.llm_creator_interface import ILlmCreator

llm_creater: ILlmCreator = AzureChatOpenAILlmCreator(cache_aside_env_values_reader, hcp_token_creator)

llm = await llm_creater.create_llm()
print(f"LLM client created: {llm}")

## Testing the LLM Integration

Now let's initialize the LLM and test it with a simple query.

Note, the "logic" is inside LlmWorker.  You can go edit the code in LlmWorker to experiment.

In [ ]:
from llm_worker import LlmWorker

resp = await LlmWorker.invoke_llm(llm, "")

In [ ]:
from fx_ai_reusables.llm.reporters.llm_reporter import LlmReporter


# available keys in the response message
await LlmReporter.show_keys(resp)

# Resp is an instance of AIMessage(langchain)
await LlmReporter.show_type(resp)


## Display the Response

Print the content of the response from the LLM.

In [ ]:
# Display the content of the response
from fx_ai_reusables.llm.reporters.llm_reporter import LlmReporter


# available keys in the response message
await LlmReporter.show_content(resp)


#show usage costs
await LlmReporter.show_usage_costs(resp)
